## 사용 패키지

In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor

In [3]:
from pycaret.regression import *

## 데이터 로드

In [489]:
# 기존 데이터 Load

train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')
submission = pd.read_csv('./data/sample_submission.csv')

In [491]:
train['현재'] = train['본사정원수'] - train['본사휴가자수'] - train['본사출장자수'] - train['현본사소속재택근무자수']

In [492]:
train.corr()

,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,중식계,석식계,현재
본사정원수,1.000000,0.244666,-0.061688,0.146544,0.452293,-0.115529,-0.173852,0.503771
본사휴가자수,0.244666,1.000000,-0.273318,-0.230789,0.065397,-0.391975,-0.316894,-0.552661
본사출장자수,-0.061688,-0.273318,1.000000,-0.308757,-0.352643,-0.512680,-0.188164,0.136389
본사시간외근무명령서승인건수,0.146544,-0.230789,-0.308757,1.000000,0.156792,0.535611,0.571168,0.309118
현본사소속재택근무자수,0.452293,0.065397,-0.352643,0.156792,1.000000,0.076509,-0.057534,-0.150393
중식계,-0.115529,-0.391975,-0.512680,0.535611,0.076509,1.000000,0.508287,0.286810
석식계,-0.173852,-0.316894,-0.188164,0.571168,-0.057534,0.508287,1.000000,0.172373
현재,0.503771,-0.552661,0.136389,0.309118,-0.150393,0.286810,0.172373,1.000000


In [496]:
test.head()

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴
0,2021-01-27,수,2983,88,182,5,358.0,모닝롤/연유버터베이글 우유/주스 계란후라이/찐계란 단호박죽/흑미밥 우거지국 고기완자...,쌀밥/흑미밥/찰현미밥 대구지리 매운돈갈비찜 오꼬노미계란말이 상추무침 포기김치 양상추...,흑미밥 얼큰순두부찌개 쇠고기우엉볶음 버섯햄볶음 (New)아삭이고추무절임 포기김치
1,2021-01-28,목,2983,104,212,409,348.0,모닝롤/대만샌드위치 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 황태국 시래기지짐 ...,쌀밥/보리밥/찰현미밥 우렁된장찌개 오리주물럭 청양부추전 수제삼색무쌈 겉절이김치 양상...,충무김밥 우동국물 오징어무침 꽃맛살샐러드 얼갈이쌈장무침 석박지
2,2021-01-29,금,2983,270,249,0,294.0,모닝롤/핫케익 우유/주스 계란후라이/찐계란 오곡죽/흑미밥 매생이굴국 고구마순볶음 양...,쌀밥/흑미밥/찰현미밥 팽이장국 수제돈까스*소스 가자미조림 동초나물무침 포기김치 양상...,흑미밥 물만둣국 카레찜닭 숯불양념꼬지어묵 꼬시래기무침 포기김치
3,2021-02-01,월,2924,108,154,538,322.0,모닝롤/촉촉한치즈케익 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 두부김칫국 새우완...,쌀밥/흑미밥/찰현미밥 배추들깨국 오리대패불고기 시금치프리타타 부추고추장무침 포기김치...,흑미밥 동태탕 돈육꽈리고추장조림 당면채소무침 모자반무침 포기김치
4,2021-02-02,화,2924,62,186,455,314.0,모닝롤/토마토샌드 우유/주스 계란후라이/찐계란 채소죽/흑미밥 호박맑은국 오이생채 양...,쌀밥/팥밥/찰현미밥 부대찌개 닭살데리야끼조림 버섯탕수 세발나물무침 알타리김치/사과푸...,흑미밥 바지락살국 쇠고기청경채볶음 두부구이*볶은김치 머위된장무침 백김치


In [500]:
test_lunch.head()

,일자,요일,본사정원수,본사휴가자수,본사출장자수,현본사소속재택근무자수,밥,국,메인
0,2021-01-27,수,2983,88,182,358.0,밥,대구지리,매운돈갈비찜
1,2021-01-28,목,2983,104,212,348.0,밥,우렁된장찌개,오리주물럭
2,2021-01-29,금,2983,270,249,294.0,밥,팽이장국,수제돈까스*소스
3,2021-02-01,월,2924,108,154,322.0,밥,배추들깨국,오리대패불고기
4,2021-02-02,화,2924,62,186,314.0,밥,부대찌개,닭살데리야끼조림


In [498]:
# 식단 전처리 데이터 Load

train_lunch = pd.read_csv('train_lunch.csv')
test_lunch = pd.read_csv('test_lunch.csv')
train_dinner = pd.read_csv('train_dinner.csv')
test_dinner = pd.read_csv('test_dinner.csv')

train_lunch.drop(columns = ['Unnamed: 0'], inplace = True)
test_lunch.drop(columns = ['Unnamed: 0'], inplace = True)
train_dinner.drop(columns = ['Unnamed: 0'], inplace = True)
test_dinner.drop(columns = ['Unnamed: 0'], inplace = True)

train_lunch.dropna(axis = 0, inplace =True)
train_dinner.dropna(axis = 0, inplace =True)

In [486]:
train_lunch.corr()

,본사정원수,본사휴가자수,본사출장자수,현본사소속재택근무자수,중식계,강수여부,month
본사정원수,1.000000,0.245172,-0.061032,0.452558,-0.115821,0.070061,0.245583
본사휴가자수,0.245172,1.000000,-0.273964,0.065317,-0.391875,-0.012474,0.193246
본사출장자수,-0.061032,-0.273964,1.000000,-0.352956,-0.512619,-0.055989,0.210200
현본사소속재택근무자수,0.452558,0.065317,-0.352956,1.000000,0.076571,0.095892,0.033443
중식계,-0.115821,-0.391875,-0.512619,0.076571,1.000000,0.105208,-0.154429
강수여부,0.070061,-0.012474,-0.055989,0.095892,0.105208,1.000000,-0.034217
month,0.245583,0.193246,0.210200,0.033443,-0.154429,-0.034217,1.000000


In [487]:
train_dinner.corr()

,본사정원수,본사휴가자수,본사출장자수,현본사소속재택근무자수,석식계,강수여부,중식계,month
본사정원수,1.000000,0.243816,-0.060555,0.459257,-0.227671,0.065108,-0.131304,0.245014
본사휴가자수,0.243816,1.000000,-0.268217,0.064422,-0.412188,0.036338,-0.136653,0.190124
본사출장자수,-0.060555,-0.268217,1.000000,-0.353409,-0.191586,-0.089646,-0.011633,0.211243
현본사소속재택근무자수,0.459257,0.064422,-0.353409,1.000000,-0.118648,0.100153,-0.003030,0.035518
석식계,-0.227671,-0.412188,-0.191586,-0.118648,1.000000,-0.019078,0.109313,-0.144690
강수여부,0.065108,0.036338,-0.089646,0.100153,-0.019078,1.000000,-0.005405,-0.043260
중식계,-0.131304,-0.136653,-0.011633,-0.003030,0.109313,-0.005405,1.000000,-0.054330
month,0.245014,0.190124,0.211243,0.035518,-0.144690,-0.043260,-0.054330,1.000000


## 강수 데이터 load 및 전처리

### 2010~ 2020 강수 데이터 Load

In [501]:
import glob
outputs = glob.glob('진주강수량/SURFACE*.csv')

temp_data = pd.DataFrame(columns=['일시','강수량(mm)'])

for output in outputs:
    temp = pd.read_csv(output, encoding='cp949')
    temp['강수량(mm)'].fillna(0, inplace= True)
    temp_data =temp_data.append(temp[['일시','강수량(mm)']])
    

In [502]:
temp_data = temp_data.fillna(method = 'bfill')
temp_data.isnull().sum()

일시         0
강수량(mm)    0
dtype: int64

### 2021 강수 데이터 Load

In [503]:
newest = temp = pd.read_csv('진주강수량\\충무공동_강수_202101_202110.csv', encoding='cp949')

In [504]:
newest.columns=['일시','시간','강수량(mm)','월']
newest.dropna(axis = 0, inplace = True)

In [505]:
import datetime
from dateutil.relativedelta import relativedelta

newest['시간'] = newest.apply(lambda x : int(x['시간']//100), axis= 1)
newest['일시'] = newest.apply(lambda x : datetime.datetime.combine(pd.to_datetime('2020-12-31')+datetime.timedelta(days=int(x['일시'].strip()))+relativedelta(months=x['월']), datetime.time(x['시간'],0,0)),axis = 1)

In [506]:
newest.drop(columns = ['시간','월'], inplace = True)

In [507]:
newest.head()

,일시,강수량(mm)
0,2021-01-01 00:00:00,0.0
1,2021-01-01 01:00:00,0.0
2,2021-01-01 02:00:00,0.0
3,2021-01-01 03:00:00,0.0
4,2021-01-01 04:00:00,0.0


### 강수 데이터 Append

In [508]:
temp_data = temp_data.append(newest)

In [509]:
temp_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99321 entries, 0 to 2882
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   일시       99321 non-null  object 
 1   강수량(mm)  99321 non-null  float64
dtypes: float64(1), object(1)
memory usage: 2.3+ MB


### 점심 강수 데이터 추출

In [510]:
# 11시 ~ 14시 사이 강수량이 있는 경우 1

df = temp_data.copy()
df['일시'] = df['일시'].astype('datetime64[ns]')
df.set_index(df['일시'],drop=True,inplace = True)
df.drop(columns = ['일시'], inplace = True)

lunch_df = df.between_time('11:00','14:00', include_start = True, include_end = True, axis = None )
lunch_df.reset_index(drop=False,inplace = True)

lunch_df['일시'] = pd.to_datetime(lunch_df['일시']).dt.date
lunch_result = lunch_df.groupby('일시')['강수량(mm)'].aggregate(['sum']).reset_index()

lunch_result['강수여부'] = lunch_result.apply(lambda x : 1 if x['sum'] > 0 else 0,axis = 1)

In [511]:
lunch_result.columns =['일자','강수량','강수여부']
lunch_result.drop(columns = ['강수량'], inplace = True)

In [512]:
lunch_result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4138 entries, 0 to 4137
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   일자      4138 non-null   object
 1   강수여부    4138 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 64.8+ KB


### 저녁 강수 데이터 추출

In [513]:
df = temp_data.copy()

df['일시'] = df['일시'].astype('datetime64[ns]')
df.set_index(df['일시'],drop=True,inplace = True)
df.drop(columns = ['일시'], inplace = True)

dinner_df = df.between_time('17:00','20:00', include_start = True, include_end = True, axis = None )
dinner_df.reset_index(drop=False,inplace = True)

dinner_df['일시'] = pd.to_datetime(dinner_df['일시']).dt.date
dinner_result = dinner_df.groupby('일시')['강수량(mm)'].aggregate(['sum']).reset_index()

dinner_result['강수여부'] = dinner_result.apply(lambda x : 1 if x['sum'] > 0 else 0,axis = 1)

In [514]:
dinner_result.columns =['일자','강수량','강수여부']
dinner_result.drop(columns = ['강수량'], inplace = True)

In [515]:
dinner_result.sum()

강수여부    480
dtype: int64

## 데이터 전처리

1. LabelEncoding 및 MinMaxScaler
2. 휴일 전, 휴일 후 column 추가
3. 강수량 column 추가


### LabelEncoding

In [332]:
# train_lunch['요일'] = train['요일'].map({'월':0, '화':1, '수':2, '목':3, '금':4})
# test_lunch['요일'] = test['요일'].map({'월':0, '화':1, '수':2, '목':3, '금':4})

# train_dinner['요일'] = train['요일'].map({'월':0, '화':1, '수':2, '목':3, '금':4})
# test_dinner['요일'] = test['요일'].map({'월':0, '화':1, '수':2, '목':3, '금':4})

In [516]:
train_lunch['요일'] = train['요일'].astype('category')
test_lunch['요일'] = test['요일'].astype('category')

train_dinner['요일'] = train['요일'].astype('category')
test_dinner['요일'] = test['요일'].astype('category')

### MinMaxScaling(Tree구조 사용 안 할 경우)


In [251]:
# from sklearn.preprocessing import MinMaxScaler

In [314]:
# merge_train.head()

,일자,요일,본사정원수,본사휴가자수,본사출장자수,현본사소속재택근무자수,밥,국,메인,중식계
0,2016-02-01,월,2601,50,150,0.0,밥,오징어찌개,쇠불고기,1039.0
1,2016-02-02,화,2601,50,173,0.0,밥,김치찌개,가자미튀김,867.0
2,2016-02-03,수,2601,56,180,0.0,카레덮밥,팽이장국,치킨핑거,1017.0
3,2016-02-04,목,2601,104,220,0.0,밥,쇠고기무국,주꾸미볶음,978.0
4,2016-02-05,금,2601,278,181,0.0,밥,떡국,돈육씨앗강정,925.0


In [263]:
# train['본사정원수'] = train['본사정원수']-train['본사휴가자수']-train['본사출장자수']-train['현본사소속재택근무자수']
# test['본사정원수'] = test['본사정원수']-test['본사휴가자수']-test['본사출장자수']-test['현본사소속재택근무자수']

In [264]:
# train['휴일전'] = train.apply(lambda x: 1 if x['요일'] == 4 else 0 , axis = 1) 
# train['휴일후'] = train.apply(lambda x: 1 if x['요일'] == 0 else 0 , axis = 1)

# test['휴일전'] = test.apply(lambda x: 1 if x['요일'] == 4 else 0 , axis = 1) 
# test['휴일후'] = test.apply(lambda x: 1 if x['요일'] == 0 else 0 , axis = 1)

In [247]:
# train[['본사정원수','본사시간외근무명령서승인건수']] = scaler.fit_transform(train[['본사정원수', '본사시간외근무명령서승인건수']])
# test[['본사정원수', '본사시간외근무명령서승인건수']] = scaler.transform(test[['본사정원수', '본사시간외근무명령서승인건수']])

In [265]:
# train['본사정원수'].max(),train['본사정원수'].min(),test['본사정원수'].max(),test['본사정원수'].min()

(2921.0, 1372.0, 2448.0, 2068.0)

## 일자 기준 기존 데이터와 강수 데이터 Merge

In [517]:
train_lunch['일자'] = pd.to_datetime(train_lunch['일자'])
test_lunch['일자'] = pd.to_datetime(test_lunch['일자'])
train_dinner['일자'] = pd.to_datetime(train_dinner['일자'])
test_dinner['일자'] = pd.to_datetime(test_dinner['일자'])

lunch_result['일자'] = pd.to_datetime(lunch_result['일자'])
dinner_result['일자'] = pd.to_datetime(dinner_result['일자'])

In [518]:
train_lunch = pd.merge(left = train_lunch , right = lunch_result, how = "inner", on = "일자")
train_dinner = pd.merge(left = train_dinner , right = dinner_result, how = "inner", on = "일자")

test_lunch = pd.merge(left = test_lunch , right = lunch_result, how = "inner", on = "일자")
test_dinner = pd.merge(left = test_dinner , right = dinner_result, how = "inner", on = "일자")

In [519]:
train_lunch[train_lunch['요일'].isnull()]

,일자,요일,본사정원수,본사휴가자수,본사출장자수,현본사소속재택근무자수,밥,국,메인,중식계,강수여부


In [520]:
train_dinner[train_dinner['요일'].isnull()]

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,밥,국,메인,중식계,석식계,강수여부


In [468]:
# 요일을 category 데이터로 변환 시 수요일 하루 데이터가 누락되는 현상 발생. 임의로 추가

train_dinner['요일'].fillna('수', inplace = True)

In [469]:
test_dinner.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50 entries, 0 to 49
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   일자              50 non-null     datetime64[ns]
 1   요일              50 non-null     category      
 2   본사정원수           50 non-null     int64         
 3   본사휴가자수          50 non-null     int64         
 4   본사출장자수          50 non-null     int64         
 5   본사시간외근무명령서승인건수  50 non-null     int64         
 6   현본사소속재택근무자수     50 non-null     float64       
 7   밥               50 non-null     object        
 8   국               50 non-null     object        
 9   메인              50 non-null     object        
 10  강수여부            50 non-null     int64         
dtypes: category(1), datetime64[ns](1), float64(1), int64(5), object(3)
memory usage: 4.5+ KB


In [521]:
# 중식계를 석식계 예측의 input으로 설정
train_dinner['중식계'] = train_lunch['중식계']

In [522]:
# 월 일 데이터 추가

train_lunch['month'] = train_lunch['일자'].dt.month
train_lunch['day'] = train_lunch['일자'].dt.day.astype('category')

train_dinner['month'] = train_dinner['일자'].dt.month
train_dinner['day'] = train_dinner['일자'].dt.day.astype('category')

test_lunch['month'] = test_lunch['일자'].dt.month
test_lunch['day'] = test_lunch['일자'].dt.day.astype('category')

test_dinner['month'] = test_dinner['일자'].dt.month
test_dinner['day'] = test_dinner['일자'].dt.day.astype('category')

In [541]:
category_data = ['요일', '밥', '국', '메인','month', 'day']
for category in category_data:
    train_lunch[category] = train_lunch[category].astype('category').cat.codes
    train_dinner[category] = train_dinner[category].astype('category').cat.codes    
    test_lunch[category] = test_lunch[category].astype('category').cat.codes    
    test_dinner[category] = test_dinner[category].astype('category').cat.codes    

In [551]:
train_dinner.head()

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,밥,국,메인,중식계,석식계,강수여부,month,day
0,2016-02-01,3,2601,50,150,238,0.0,37,222,339,1039.0,331.0,0,1,0
1,2016-02-02,4,2601,50,173,319,0.0,103,179,333,867.0,560.0,0,1,1
2,2016-02-03,2,2601,56,180,111,0.0,37,245,431,1017.0,573.0,0,1,2
3,2016-02-04,1,2601,104,220,355,0.0,35,211,168,978.0,525.0,0,1,3
4,2016-02-05,0,2601,278,181,34,0.0,37,241,88,925.0,330.0,0,1,4


## 점심과 저녁을 식수 인원을 예측할 2개 모델 생성

In [544]:
# 점심 데이터 pycaret setup
setup_lunch = setup(data=train_lunch,target='중식계',categorical_features = category_data,ignore_features=['일자'])

,Description,Value
0,session_id,2101
1,Target,중식계
2,Original Data,"(1204, 13)"
3,Missing Values,False
4,Numeric Features,4
5,Categorical Features,7
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(842, 562)"


In [552]:
# 저녁 데이터 pycaret setup (점심 예측 모델 생성 및 예측 완료 후 사용)
setup_dinner = setup(data=train_dinner,target='석식계',categorical_features = category_data,ignore_features=['일자'])

,Description,Value
0,session_id,6221
1,Target,석식계
2,Original Data,"(1169, 15)"
3,Missing Values,False
4,Numeric Features,6
5,Categorical Features,7
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(818, 651)"


In [444]:
best = compare_models(sort = 'MAE')

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,73.2522,9850.9060,98.7089,0.7755,0.1211,0.0890,2.2170
gbr,Gradient Boosting Regressor,75.2166,10279.4269,100.7663,0.7656,0.1235,0.0913,0.3790
lightgbm,Light Gradient Boosting Machine,76.6151,10689.0482,102.3619,0.7578,0.1261,0.0929,0.2590
rf,Random Forest Regressor,77.4441,10684.5746,102.7125,0.7574,0.1276,0.0944,0.8030
et,Extra Trees Regressor,79.4306,11606.2235,106.9198,0.7359,0.1319,0.0969,1.1940
lasso,Lasso Regression,80.5501,11768.2669,108.0502,0.7322,0.1349,0.0986,0.0340
br,Bayesian Ridge,82.6268,12064.8323,109.4765,0.7244,0.1378,0.1012,0.4560
ridge,Ridge Regression,88.0557,13413.1484,115.5149,0.6919,0.1457,0.1075,0.0380
omp,Orthogonal Matching Pursuit,88.7153,14591.3422,120.2895,0.6652,0.1479,0.1074,0.0360
ada,AdaBoost Regressor,89.1647,13363.1909,115.1666,0.6954,0.1389,0.1087,0.4050


In [553]:
cbt = create_model('xgboost', cross_validation = False)

# ada = create_model('ada', cross_validation = False)
# lgbm = create_model('lightgbm', cross_validation = False)
# gbr = create_model('gbr', cross_validation = False)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,51.887299,4822.717773,69.445801,0.6446,0.5101,0.1168


## 학습

In [554]:
tuned_cbt = tune_model(cbt, optimize = 'MAE', n_iter = 10)


# tuned_gbr = tune_model(gbr, optimize = 'RMSE', n_iter = 10)
# tuned_ada = tune_model(ada, optimize = 'RMSE', n_iter = 10)
# tuned_lgbm = tune_model(lgbm, optimize = 'RMSE', n_iter = 10)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,60.4454,7445.6016,86.2879,0.5544,0.9154,0.1302
1,52.9547,4884.9937,69.8927,0.6669,0.1887,0.1383
2,44.2112,5798.3662,76.1470,0.4940,0.1457,0.0917
3,49.3482,4454.9331,66.7453,0.6066,0.1523,0.1112
4,46.8239,4063.0945,63.7424,0.5978,0.1496,0.1043
5,48.9865,4538.0469,67.3650,0.6157,0.1558,0.1150
6,45.3710,3967.9558,62.9917,0.6313,0.1286,0.0933
7,62.9037,8999.7559,94.8670,0.3766,0.7059,0.1226
8,39.2747,2600.3730,50.9939,0.7373,0.1232,0.0866
9,56.9596,8235.8545,90.7516,0.5400,0.9659,0.1328


In [287]:
# blender_specific = blend_models(estimator_list = [tuned_lgbm,tuned_ada,tuned_gbr], optimize = 'MAE')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,62.5033,8628.8110,92.8914,0.4157,0.9212,0.1090
1,75.0920,10655.1177,103.2236,0.3512,0.9260,0.1585
2,64.9317,7729.1257,87.9154,0.4641,0.6729,0.1561
3,64.7399,9772.1927,98.8544,0.2649,0.6693,0.1290
4,73.4076,10183.3260,100.9125,0.3568,0.9337,0.1766
5,77.9250,12958.8613,113.8370,0.4230,1.3074,0.1511
6,85.2366,15007.1184,122.5035,0.4020,1.4550,0.1540
7,71.1739,10865.9423,104.2398,0.4390,1.1335,0.1427
8,69.2017,10975.7554,104.7652,0.4284,1.1344,0.1237
9,65.7473,8863.3569,94.1454,0.5050,0.9286,0.1486


## 예측 및 결과 제출

In [547]:
# 중식계 예측

final_model_lunch = finalize_model(tuned_cbt)
pred1 = predict_model(final_model_lunch, data = test_lunch)

In [555]:
# 석식계 예측(dinner setup 및 모델 생성 후 사용)

test_dinner['중식계'] = pred1['Label']
final_model_dinner = finalize_model(tuned_cbt)
pred2 = predict_model(final_model_dinner, data = test_dinner)

In [556]:
submission['중식계'] = pred1['Label']
submission['석식계'] = pred2['Label']

In [456]:
submission.head()

,일자,중식계,석식계
0,2021-01-27,1041.003129,391.390526
1,2021-01-28,922.015795,455.878211
2,2021-01-29,653.542934,268.101406
3,2021-02-01,1226.908857,605.436209
4,2021-02-02,1022.845271,489.591339


In [557]:
submission.to_csv('baseline_xgboost_1103.csv', index=False)